# Time Series Forecasting Model Evaluation

### Common Imports

In [ ]:
from sklearn.model_selection import train_test_split

from helpers import *
from abstract import ForecastingMethod
import numpy as np

### Shared Parameters

In [ ]:
TRAIN_TEST_RATIO = 8 / 10
START_DATE = date(2020, 1, 1)

### Training & Test Data Preparation

In [ ]:
# Preprocess the data: apply normalization and scaling like the paper does
new_filepath, min_value, max_value, initial_value = (
    ForecastingMethod.preprocess_data(
        filepath="../data/paper-data.csv",
        difference=False,
        scale_to_range=True,
        training_data_cutoff=TRAIN_TEST_RATIO,
    )
)

# Load the data
X, y = ForecastingMethod.load_data(filepath=new_filepath)
train_test_split_index = int(len(X) * TRAIN_TEST_RATIO)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=(1 - TRAIN_TEST_RATIO), shuffle=False
)

# Linear Regression

In [ ]:
from linear_regression import LinearRegression

### Training - Model Creation & Weight Finding

In [ ]:
models = [
    (
        "Linear Regression",
        LinearRegression(
            regularization=None, fit_intercept=False, cv_folds=None
        ),
    ),
    (
        "Linear Regression with Y-Intercept",
        LinearRegression(
            regularization=None, fit_intercept=True, cv_folds=None
        ),
    ),
    (
        "L1 Regularized Linear Regression",
        LinearRegression(
            regularization="l1",
            fit_intercept=True,
            cv_folds=5,
            alphas=[0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 1, 5, 10, 100],
        ),
    ),
    (
        "L2 Regularized Linear Regression",
        LinearRegression(
            regularization="l2",
            fit_intercept=True,
            cv_folds=5,
            alphas=[
                0.0001,
                0.001,
                0.01,
                0.1,
                0.3,
                0.5,
                1,
                2,
                3,
                4,
                5,
                6,
                7,
                10,
                100,
            ],
        ),
    ),
]

for name, model in models:
    model.train(X_train, y_train)

### Prediction - Timeseries Forecasting & Evaluation

In [ ]:
for name, model in models:
    print("#" * 50)
    print(name)
    print("#" * 50)
    print(f"Training Loss (MSE): {model.score(X_train, y_train)}")
    print(f"Testing Loss (MSE): {model.score(X_test, y_test)}")
    if "alpha" in model.model.get_params():
        print(f"Best Alpha: {model.model.get_params()["alpha"]}")

In [ ]:
for name, model in models:
    # Post process X
    X_pred_scaled = ForecastingMethod.post_process_data(
        model.predict(X),
        min_value,
        max_value,
        initial_value,
        scale_to_range=True,
    )
    # Post process y
    y_scaled = ForecastingMethod.post_process_data(
        y, min_value, max_value, initial_value, scale_to_range=True
    )

    plot_sales_growth(
        START_DATE,
        y_scaled,
        X_pred_scaled,
        pred_linestyle="-.",
        pred_color="lime",
        title=name,
        train_test_split=train_test_split_index,
    )

# PQC

In [ ]:
from pqc import PQC

### Training - Model Creation & Weight Finding

In [ ]:
N_WIRES = 12
N_LAYERS = 2

# Create L-BFGS-B model
pqc_model_lbfgsb = PQC(
    n_wires=N_WIRES, n_layers=N_LAYERS, optimizer="L-BFGS-B"
)
# Create the COBYLA model
pqc_model_cobyla = PQC(n_wires=N_WIRES, n_layers=N_LAYERS, optimizer="COBYLA")

# load weights - since training takes 30+ minutes
pqc_model_lbfgsb.load_weights("../weights/pqc_lbfgsb.npy")
pqc_model_cobyla.load_weights("../weights/pqc_cobyla.npy")

### Prediction - Timeseries Forecasting & Evaluation

In [ ]:
# Predict the dataset as whole
predictions_L = pqc_model_lbfgsb.predict(X)
predictions_C = pqc_model_cobyla.predict(X)

# Compute the training loss with processed data
train_mse_L = np.mean((predictions_L[:train_test_split_index] - y_train) ** 2)
train_mse_C = np.mean((predictions_C[:train_test_split_index] - y_train) ** 2)

# Compute testing loss
test_mse_L = np.mean((predictions_L[train_test_split_index:] - y_test) ** 2)
test_mse_C = np.mean((predictions_C[train_test_split_index:] - y_test) ** 2)

print("#" * 50)
print("PQC with L-BFGS-B Optimizer")
print("#" * 50)
print(f"Training Loss (MSE): {train_mse_L}")
print(f"Test Loss (MSE): {test_mse_L}")
print("#" * 50)
print("PQC with COBYLA Optimizer")
print("#" * 50)
print(f"Training Loss (MSE): {train_mse_C}")
print(f"Test Loss (MSE): {test_mse_C}")

In [ ]:
# Post process the predictions
predictions_L_scaled = ForecastingMethod.post_process_data(
    predictions_L, min_value, max_value, initial_value, scale_to_range=True
)
predictions_C_scaled = ForecastingMethod.post_process_data(
    predictions_C, min_value, max_value, initial_value, scale_to_range=True
)
# Post process y
y_scaled = ForecastingMethod.post_process_data(
    y, min_value, max_value, initial_value, scale_to_range=True
)

# Plot the results
plot_sales_growth(
    START_DATE,
    y_scaled,
    predictions_L_scaled,
    pred_linestyle="--",
    pred_color="blue",
    title="PQC L-BFGS-B",
    train_test_split=train_test_split_index,
)
plot_sales_growth(
    START_DATE,
    y_scaled,
    predictions_C_scaled,
    pred_linestyle="--",
    pred_color="red",
    title="PQC COBYLA",
    train_test_split=train_test_split_index,
)

# Neural Network

In [ ]:
from neural_network import NeuralNetwork

### Training - Model Creation & Weight Finding

In [ ]:
nn = NeuralNetwork()
nn.train(X_train, y_train)

### Prediction - Timeseries Forecasting & Evaluation

In [ ]:
predictions = nn.predict(X)

train_mse = np.mean((predictions[:train_test_split_index] - y_train) ** 2)
test_mse = np.mean((predictions[train_test_split_index:] - y_test) ** 2)

print(f"Training Loss (MSE): {train_mse}")
print(f"Testing Loss (MSE): {test_mse}")

In [ ]:
# Post process X
X_pred_scaled = ForecastingMethod.post_process_data(
    predictions, min_value, max_value, initial_value, scale_to_range=True
)
# Post process y
y_scaled = ForecastingMethod.post_process_data(
    y, min_value, max_value, initial_value, scale_to_range=True
)

plot_sales_growth(
    START_DATE,
    y_scaled,
    X_pred_scaled,
    pred_linestyle=":",
    pred_color="firebrick",
    title="Neural Network",
    train_test_split=train_test_split_index,
)